In [1]:
import pandas as pd
import sqlite3

In [3]:
conn = sqlite3.connect("restaurants.db")
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [4]:
orders = pd.read_csv("orders.csv")
users = pd.read_json("users.json")

In [5]:
final_dataset = (
    orders
    .merge(users, on="user_id", how="left")
    .merge(restaurants, on="restaurant_id", how="left")
)

In [7]:
final_dataset = final_dataset.assign(
    order_date=pd.to_datetime(final_dataset['order_date'], dayfirst=True),
    total_amount=pd.to_numeric(final_dataset['total_amount'], errors='coerce'),
    rating=pd.to_numeric(final_dataset['rating'], errors='coerce')
)

In [19]:
final_dataset[final_dataset['membership']=='Gold'] \
.groupby('city')['total_amount'].sum().idxmax()

'Chennai'

In [9]:
final_dataset.groupby('cuisine')['total_amount'].mean().idxmax()

'Mexican'

In [10]:
final_dataset.groupby('user_id')['total_amount'].sum().gt(1000).sum()

np.int64(2544)

In [11]:
final_dataset['rating_range'] = pd.cut(
    final_dataset['rating'],
    [3.0, 3.5, 4.0, 4.5, 5.0]
)
final_dataset.groupby('rating_range')['total_amount'].sum().idxmax()

Interval(4.5, 5.0, closed='right')

In [12]:
final_dataset[final_dataset['membership']=='Gold'] \
.groupby('city')['total_amount'].mean().idxmax()

'Chennai'

In [13]:
final_dataset.groupby('cuisine').agg(
    restaurants=('restaurant_id','nunique'),
    revenue=('total_amount','sum')
).sort_values(['restaurants','revenue'], ascending=[True, False])

,restaurants,revenue
cuisine,,
Chinese,120,1930504.65
Italian,126,2024203.80
Indian,126,1971412.58
Mexican,128,2085503.09


In [14]:
round(
    (final_dataset['membership']=='Gold').mean() * 100
)

50

In [17]:
final_dataset.groupby(['membership','cuisine'])['total_amount'] \
.sum().idxmax()

('Regular', 'Mexican')

In [16]:
final_dataset['quarter'] = final_dataset['order_date'].dt.quarter
final_dataset.groupby('quarter')['total_amount'].sum().idxmax()

np.int32(3)

In [24]:
final_dataset.groupby('restaurant_id')['total_amount'] \
.agg(['count','mean']) \
.query('count < 20') \
.sort_values('mean', ascending=False) \
.head(1)


,count,mean
restaurant_id,,
294,13,1040.222308


In [26]:
(final_dataset['membership'] == 'Gold').sum()

np.int64(4987)

In [27]:
round(final_dataset[final_dataset['city'] == 'Hyderabad']['total_amount'].sum())

1889367

In [28]:
final_dataset['user_id'].nunique()

2883

In [30]:
round(
    final_dataset[final_dataset['membership'] == 'Gold']['total_amount'].mean(),2
    
)

np.float64(797.15)

In [31]:
(final_dataset['rating'] >= 4.5).sum()

np.int64(3374)

In [32]:
top_city = (
    final_dataset[final_dataset['membership'] == 'Gold']
    .groupby('city')['total_amount']
    .sum()
    .idxmax()
)

final_dataset[
    (final_dataset['membership'] == 'Gold') &
    (final_dataset['city'] == top_city)
].shape[0]


1337